In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb

# Load the dataset
train_data = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

# Preprocess the data
features = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
X = train_data[features]
y = train_data['Transported']

# Handling categorical and numerical variables
numerical_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
categorical_cols = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

# Define the model
model_rf = RandomForestClassifier(n_estimators=100, random_state=0)
model_xgb = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Create and evaluate the pipeline
pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model_rf)])

pipeline_xgb = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model_xgb)])

# Split data into train and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Train and evaluate the Random Forest model
pipeline_rf.fit(X_train, y_train)
preds_valid_rf = pipeline_rf.predict(X_valid)
print('Random Forest accuracy:', accuracy_score(y_valid, preds_valid_rf))

# Train and evaluate the XGBoost model
pipeline_xgb.fit(X_train, y_train)
preds_valid_xgb = pipeline_xgb.predict(X_valid)
print('XGBoost accuracy:', accuracy_score(y_valid, preds_valid_xgb))


Random Forest accuracy: 0.7757331799884991
XGBoost accuracy: 0.7780333525014376


In [3]:
# Create and evaluate the pipeline
pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model_rf)])

# Train the Random Forest model on the training data
pipeline_rf.fit(X, y)  # Note: Use the full training data X and labels y here

# Now we use the pipeline to predict on the test data, which will ensure the same preprocessing steps
test_preds_rf = pipeline_rf.predict(test_data[features])

# Create a DataFrame for the submission file
submission = pd.DataFrame({
    'PassengerId': test_data['PassengerId'],
    'Transported': test_preds_rf
})

# Convert 'Transported' from boolean to the required format (True/False as strings)
submission['Transported'] = submission['Transported'].astype(str)

# Save the submission file
submission.to_csv('submission.csv', index=False)

print("Submission file saved.")

Submission file saved.
